In [ ]:
'''sprint-4 

Train thd model in IBM cloud 
prediction based on scoring response of Trained model at IBM CLOUD
'''

In [2]:
import numpy as np
from flask import Flask, request, jsonify, render_template
import joblib
import requests

# NOTE: you must manually set API_KEY below using information retrieved from your IBM Cloud account.
API_KEY = "eq_uB9S4u70ys6t-jeWs2uoNIscWVl0Gh0C9GTKlDloW"
token_response = requests.post('https://iam.cloud.ibm.com/identity/token', data={"apikey":
 API_KEY, "grant_type": 'urn:ibm:params:oauth:grant-type:apikey'})
mltoken = token_response.json()["access_token"]

header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}


app = Flask(__name__)
#model = joblib.load('xg_RFR_forecast_model.sav')
#model = joblib.load('dec_model.sav')

@app.route('/')
def home():
    return render_template('intro.html')
@app.route('/predict')
def predict():
    return render_template('predict_page.html')
@app.route('/windapi',methods=['POST'])
def windapi():
    city=request.form.get('city')
    apikey="e26bb531d3393dec23475ee08ea9559b"
    url="http://api.openweathermap.org/data/2.5/weather?q="+city+"&appid="+apikey
    resp = requests.get(url)
    resp=resp.json()
    temp = str((resp["main"]["temp"])-273.15) +" °C"
    humid = str(resp["main"]["humidity"])+" %"
    pressure = str(resp["main"]["pressure"])+" mmHG"
    speed = str((resp["wind"]["speed"])*3.6)+" Km/hr"
    direc = str((resp["wind"]["deg"]))+" deg"
    return render_template('predict_page.html', temp=temp, humid=humid, pressure=pressure,speed=speed,direction = direc)   
@app.route('/y_predict',methods=['POST'])
def y_predict():
    x_test =[]
    month_dic = {"jan":1,"feb":2,"mar":3,"apr":4,"may":5,"jun":6,"jul":7,"aug":8,"sept":9,"oct":10,"nov":11,"dec":12}
    m = request.form['month']
    mon = month_dic[m]
    print(m)
    #x_test.append(month_dic[m])
    d = int(request.form['day'])
    #x_test.append(d)
    t = float(request.form['temp'])
    #x_test.append(t)
    direc = float(request.form['direc'])
    #x_test.append(direc)
    speed= float(request.form['wind'])
    #x_test.append(speed)
    #x_test = [x_test]
    x_test =[[mon,d,t,direc,speed]]
    payload_scoring = {"input_data": [{"field": [['m','d','t','direc','speed']], "values": x_test}]}

    response_scoring = requests.post('https://us-south.ml.cloud.ibm.com/ml/v4/deployments/3c935904-17f3-4b91-993e-7fbf112afe46/predictions?version=2022-11-06', json=payload_scoring,headers={'Authorization': 'Bearer ' + mltoken})
    print("Scoring response")
    print(response_scoring.json())

    predictions = response_scoring.json()
    output = predictions['predictions'][0]['values'][0][0]
    print("final prediction",output)
    
    
    
    print(x_test)
    
    return render_template('predict_page.html', prediction_text='The energy predicted is {:.2f} KWh'.format(output))


if __name__ == "__main__":
    app.run(debug=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Nov/2022 16:56:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2022 16:56:36] "GET /static/images/m123.gif HTTP/1.1" 404 -
127.0.0.1 - - [07/Nov/2022 16:56:40] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2022 16:56:40] "GET /static/css/main.css HTTP/1.1" 404 -
127.0.0.1 - - [07/Nov/2022 16:56:40] "GET /static/css/media.css HTTP/1.1" 404 -
127.0.0.1 - - [07/Nov/2022 16:56:40] "GET /static/css/items_grid.css HTTP/1.1" 404 -
127.0.0.1 - - [07/Nov/2022 16:56:46] "POST /windapi HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2022 16:56:47] "GET /static/css/main.css HTTP/1.1" 404 -
127.0.0.1 - - [07/Nov/2022 16:56:47] "GET /static/css/items_grid.css HTTP/1.1" 404 -
127.0.0.1 - - [07/Nov/2022 16:56:47] "GET /static/css/media.css HTTP/1.1" 404 -


jun


127.0.0.1 - - [07/Nov/2022 16:57:10] "POST /y_predict HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2022 16:57:10] "GET /static/css/media.css HTTP/1.1" 404 -
127.0.0.1 - - [07/Nov/2022 16:57:10] "GET /static/css/items_grid.css HTTP/1.1" 404 -
127.0.0.1 - - [07/Nov/2022 16:57:10] "GET /static/css/main.css HTTP/1.1" 404 -


Scoring response
{'predictions': [{'fields': ['prediction'], 'values': [[367.0]]}]}
final prediction 367.0
[[6, 30, 33.99, 0.0, 3.708]]
